In [1]:
import os
%pwd
os.chdir("../")
%pwd

'c:\\Users\\surjy\\OneDrive\\Desktop\\chest_cancer.py'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [3]:
from cnnclassifier.constants import *
from cnnclassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [5]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [6]:
from typing import Self
import tensorflow as tf
from pathlib import Path

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

        # Recompile the model to fix any optimizer mismatch issues
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [7]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-08-25 22:27:31,486: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-25 22:27:31,500: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-25 22:27:31,502: INFO: common: created directory at: artifacts]
[2024-08-25 22:27:31,502: INFO: common: created directory at: artifacts\training]
[2024-08-25 22:27:31,820: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 321 images belonging to 4 classes.
Found 1292 images belonging to 4 classes.
Epoch 1/200


c:\Users\surjy\OneDrive\Desktop\chest_cancer.py\chestcancer\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


80/80 ━━━━━━━━━━━━━━━━━━━━ 180s 2s/step - accuracy: 0.4375 - loss: 1.7878 - val_accuracy: 0.5562 - val_loss: 0.9642
Epoch 2/200
 1/80 ━━━━━━━━━━━━━━━━━━━━ 1:52 1s/step - accuracy: 0.5833 - loss: 1.2582

C:\Users\surjy\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5833 - loss: 1.2582 - val_accuracy: 0.0000e+00 - val_loss: 3.8710
Epoch 3/200
80/80 ━━━━━━━━━━━━━━━━━━━━ 182s 2s/step - accuracy: 0.4840 - loss: 1.1853 - val_accuracy: 0.4375 - val_loss: 1.2157
Epoch 4/200
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7500 - loss: 0.7913 - val_accuracy: 0.0000e+00 - val_loss: 5.0633
Epoch 5/200
80/80 ━━━━━━━━━━━━━━━━━━━━ 222s 3s/step - accuracy: 0.5047 - loss: 1.3278 - val_accuracy: 0.5312 - val_loss: 1.0473
Epoch 6/200
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5000 - loss: 0.7036 - val_accuracy: 0.0000e+00 - val_loss: 4.4735
Epoch 7/200
80/80 ━━━━━━━━━━━━━━━━━━━━ 201s 3s/step - accuracy: 0.4622 - loss: 1.2768 - val_accuracy: 0.4000 - val_loss: 1.2873
Epoch 8/200
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5000 - loss: 1.1340 - val_accuracy: 0.0000e+00 - val_loss: 4.2053
Epoch 9/200
80/80 ━━━━━━━━━━━━━━━━━━━━ 191s 2s/step - accuracy: 0.5031 - loss: 1.3928 - val_accuracy: 0.